In [ ]:
!apt-get update -y
!apt-get install -y build-essential
!pip install -q tree_sitter boto3 smart_open[s3] datasets
!pip install -U fsspec datasets aiohttp

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,901 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,546 kB]
Get:13 http://archive.ubuntu.com/ub

In [ ]:
!git clone https://github.com/bigcode-project/starcoder2-self-align.git
%cd starcoder2-self-align/seed_gathering

Cloning into 'starcoder2-self-align'...
remote: Enumerating objects: 477, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 477 (delta 84), reused 90 (delta 70), pack-reused 357 (from 1)
Receiving objects: 100% (477/477), 273.31 KiB | 10.12 MiB/s, done.
Resolving deltas: 100% (254/254), done.
/content/starcoder2-self-align/seed_gathering


In [ ]:
# STEP 1: Uninstall incorrect tree_sitter
!pip uninstall -y tree_sitter

# STEP 2: Install correct official version
!pip install tree-sitter==0.20.1

# STEP 3: Remove old grammar if it exists
!rm -rf tree-sitter-c-sharp

# STEP 4: Clone C# grammar correctly (with submodules!)
!git clone --recurse-submodules https://github.com/tree-sitter/tree-sitter-c-sharp.git

# STEP 5: Build the parser shared object
from tree_sitter import Language, Parser

# Build the C# grammar into a shared library (this creates 'build/cs-lang.so')
Language.build_library(
    'build/cs-lang.so',  # Output path for the compiled library
    ['tree-sitter-c-sharp']  # Path to the cloned C# grammar
)

# Load the C# language from the built library
C_SHARP_LANGUAGE = Language('build/cs-lang.so', 'c_sharp')

# Initialize a new parser for C# language
parser = Parser()
parser.set_language(C_SHARP_LANGUAGE)

Found existing installation: tree-sitter 0.24.0
Uninstalling tree-sitter-0.24.0:
  Successfully uninstalled tree-sitter-0.24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tree-sitter: filename=tree_sitter-0.20.1-cp311-cp311-linux_x86_64.whl size=425846 sha256=025aeca10afd19f56dd96828d53b21900a52e02af562f4f9086f78b5c4e29c32
  Stored in directory: /root/.cache/pip/wheels/6c/be/3a/6841c52111041475b3c693b347ac03f305330cbf7fb1c6365d
Successfully built tree-sitter
Cloning into 'tree-sitter-c-sharp'...
remote: Enumerating objects: 3966, done.
remote: Counting objects: 100% (845/845), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 3966 (delta 772), reused 736 (delta 724), pack-reused 3121 (from 3)
Receiving objects: 100% (3966/3966), 122.11 MiB | 17.84 MiB/s, done.
Resolving deltas

In [ ]:
import sys
sys.path.append('/content/starcoder2-self-align/seed_gathering')

In [ ]:
!unzip /content/datasets-20250514T224535Z-1-001.zip -d /content/

Archive:  /content/datasets-20250514T224535Z-1-001.zip
  inflating: /content/datasets/seed2/dataset_info.json  
  inflating: /content/datasets/seed2/state.json  
  inflating: /content/datasets/seed2/data-00000-of-00001.arrow  


In [ ]:
from datasets import Dataset

# Load the Arrow file
arrow_dataset = Dataset.from_file("/content/datasets/seed2/data-00000-of-00001.arrow")

# Save it as a Parquet file
arrow_dataset.to_parquet("/content/seed2_output.parquet")

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

2032148

In [ ]:
import pandas as pd

df = pd.read_parquet("/content/seed2_output.parquet")

# Rename 'content' to 'seed'
df.rename(columns={"content": "seed"}, inplace=True)

# Drop 'sha1' column if it exists
df.drop(columns=["sha1"], errors="ignore", inplace=True)

# Reset 'id' to start from 0
df["id"] = range(1, len(df) + 1)

df.to_parquet("/content/seed2_output_cleaned.parquet", index=False)

In [ ]:
import pandas as pd

# Load the Parquet file
df = pd.read_parquet("/content/seed2_output_cleaned.parquet")

# Show the first 5 rows
print(df.head())

                                                seed  id
0  ///<summary>\n/// Returns a deep copy of the c...   1
1  /// <summary>\n/// 初始化任务集合\n/// </summary>\n//...   2
2  /// <summary>\n/// Tries to convert the string...   3
3  /// <summary>\n/// Returns the concrete implem...   4
4  /// <summary>\n/// Gets the positive modulo of...   5


In [ ]:
%cd /content/starcoder2-self-align/src/star_align

/content/starcoder2-self-align/src/star_align


In [ ]:
!cp /content/starcoder2-self-align/src/star_align/utils.py /content/

In [ ]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [ ]:
#run in terminal
'''
python -m vllm.entrypoints.openai.api_server \
    --model bigcode/starcoder2-15b \
    --host 0.0.0.0 \
    --port 8000
'''

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "EMPTY"
os.environ["OPENAI_BASE_URL"] = "http://0.0.0.0:8000/v1"

In [ ]:
MAX_NEW_DATA=1000000

!python /content/starcoder2-self-align/src/star_align/self_ossinstruct.py \
  --seed_data_files "/content/seed2_output_cleaned.parquet" \
  --use_vllm_server True \
  --instruct_mode "S->C" \
  --max_new_data $MAX_NEW_DATA \
  --tag concept_gen \
  --temperature 0.7 \
  --seed_code_start_index 0 \
  --model bigcode/starcoder2-15b \
  --num_fewshots 8 \
  --num_batched_requests 500 \
  --num_sample_per_request 1 \
  --async_micro_batch_size 20 \
  --delay 0

Streaming output truncated to the last 5000 lines.
                    allAddedRows.AddRange(addedRows);
                }
            }
            if ((this._ventaTableAdapter != null)) {
                global::System.Data.DataRow[] addedRows = dataSet.venta.Select(null, null, global::System.Data.DataViewRowState.Added);
                if (((addedRows != null) 
                            && (0 < addedRows.Length))) {
                    result = (result + this._ventaTableAdapter.Update(addedRows));
                    allAddedRows.AddRange(addedRows);
                }
            }
            if ((this._detalle_ventaTableAdapter != null)) {
                global::System.Data.DataRow[] addedRows = dataSet.detalle_venta.Select(null, null, global::System.Data.DataViewRowState.Added);
                if (((addedRows != null) 
                            && (0 < addedRows.Length))) {
                    result = (result + this._detalle_ventaTableAdapter.Update(addedRows));
         

In [ ]:
MAX_NEW_DATA=1000000

!python /content/starcoder2-self-align/src/star_align/self_ossinstruct.py \
  --seed_data_files "/content/starcoder2-self-align/src/star_align/data-concept_gen-s_c-78ce2-0-20250515_045332.jsonl" \
  --use_vllm_server True \
  --instruct_mode "C->I" \
  --max_new_data $MAX_NEW_DATA \
  --tag concept_gen \
  --temperature 0.7 \
  --seed_code_start_index 0 \
  --model bigcode/starcoder2-15b \
  --num_fewshots 8 \
  --num_batched_requests 500 \
  --num_sample_per_request 1 \
  --async_micro_batch_size 20 \
  --delay 0

Streaming output truncated to the last 5000 lines.

### Task
Implement a Python program that initializes a NumPy array randomly (0--1) and segments it into clusters based on provided breakpoints. Each cluster is defined by consecutive elements between two breakpoints. The output should be a list of NumPy arrays representing these clusters.

For example, given a size 10 and breakpoints `[3, 6, 8]`, the program should first initialize a NumPy array of 10 random elements between 0 and 1 and then output three clusters: elements at indices `0-2`, `3-5`, `6-7`, and `8-9`. Ensure that the program handles cases where breakpoints are at the boundaries of the array or if there are no breakpoints, in which case the entire array should be a single cluster. Include error handling to manage invalid inputs such as negative array sizes or breakpoints that are out of range.

## Example 2
### Properties
category: function implementation
language: C//
difficulty: easy
concepts: 2-pointer technique, condi

In [ ]:
MAX_NEW_DATA=1000000

!python /content/starcoder2-self-align/src/star_align/self_ossinstruct.py \
  --seed_data_files "/content/starcoder2-self-align/src/star_align/data-concept_gen-c_i-f4b55-0-20250515_051008.jsonl" \
  --use_vllm_server True \
  --instruct_mode "I->R" \
  --max_new_data $MAX_NEW_DATA \
  --tag concept_gen \
  --temperature 0.7 \
  --seed_code_start_index 0 \
  --model bigcode/starcoder2-15b \
  --num_fewshots 8 \
  --num_batched_requests 500 \
  --num_sample_per_request 1 \
  --async_micro_batch_size 20 \
  --delay 0

Streaming output truncated to the last 5000 lines.
    encoded_chars = []
    while value > 0:
        chunk = (value & 31) + 63
        value >>= 5
        if value > 0:
            chunk |= 32
        encoded_chars.append(chr(chunk))

    return ''.join(reversed(encoded_chars))
```

Keep in mind that this function is designed for non-negative integers. In Python, right-shifting (`>>`) a negative integer preserves the sign bit, which could lead to infinite loops or other incorrect behaviors if the same encoding approach is applied to negative integers without appropriate adjustments.
</response>

<tests>
You can validate this function with the following test cases that cover scenarios from zero to a number requiring more than one chunk, ensuring the function operates as expected:

```csharp
// assert encode_integer_to_chars(0) == chr(63)
// assert encode_integer_to_chars(1) == chr(63 + 1)
// assert encode_integer_to_chars(31) == chr(63 + 31)
// assert encode_integer_to_chars(0b1)) == 

In [ ]:
import json
import re

# 🔧 Path to your I→R output file
file_path = "/content/starcoder2-self-align/src/star_align/data-concept_gen-i_r-f14f6-0-20250515_052621.jsonl"

# === Extract C# code sections from Markdown-style or XML-style tags ===
def extract_csharp_sections(text):
    # Match ```csharp ... ``` or <response>...</response>, <tests>...</tests>
    response_code = None
    test_code = None

    # Markdown-style fenced code blocks
    blocks = re.findall(r"```csharp(.*?)```", text, re.DOTALL | re.IGNORECASE)
    if len(blocks) >= 2:
        response_code = blocks[0].strip()
        test_code = blocks[1].strip()
    else:
        # Fallback to <response> and <tests> tags
        resp_match = re.search(r"<response>\s*(.*?)\s*</response>", text, re.DOTALL)
        test_match = re.search(r"<tests>\s*(.*?)\s*</tests>", text, re.DOTALL)
        if resp_match and test_match:
            response_code = resp_match.group(1).strip()
            test_code = test_match.group(1).strip()

    return response_code, test_code

# === Process JSONL file ===
total_pairs = 0
valid_blocks = 0
missing_blocks = 0
examples = []

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        text = obj.get("response", "")
        func_code, test_code = extract_csharp_sections(text)

        total_pairs += 1
        if func_code and test_code:
            valid_blocks += 1
            if len(examples) < 3:
                examples.append((func_code[:300], test_code[:300]))  # Truncate long output
        else:
            missing_blocks += 1

# === Print result summary ===
print(f"✅ Total responses checked:     {total_pairs}")
print(f"📌 Valid C# blocks (code + test): {valid_blocks}")
print(f"⚠️ Missing or incomplete pairs: {missing_blocks}")

# === Show example extracted blocks ===
for i, (func, test) in enumerate(examples):
    print(f"\n--- Example {i+1} ---\n[Function Snippet]:\n{func}\n[Test Snippet]:\n{test}")

✅ Total responses checked:     978
📌 Valid C# blocks (code + test): 942
⚠️ Missing or incomplete pairs: 36

--- Example 1 ---
[Function Snippet]:
public static Matrix Inverse(Matrix matrix)
{
    try
    {
        return Math.Inverse(matrix);
    }
    catch (SingularMatrixException e)
    {
        throw new ArgumentException("Matrix is not invertible.", e);
    }
}
[Test Snippet]:
// Test valid matrix inversion
matrix = [
    [1, 2],
    [3, 4]
];
expected_inverse = [
    [0.4, -0.2],
    [-0.3, 0.1]
];
// assert Inverse(matrix) == expected_inverse

// Test invalid matrix inversion
matrix = [
    [0, 0],
    [0, 0]
];
try:
    Inverse(matrix);
    // assert False, "Failed to 

--- Example 2 ---
[Function Snippet]:
public static IEnumerable<Type> ExtractPublicTypes(Assembly assembly)
{
    Type[] types = assembly.GetTypes();
    return types.Where(t => t.IsPublic && t.IsAssignableFrom(typeof(IEnumerable<>)));
}
[Test Snippet]:
assembly = Assembly.Load("System.Collections")
// assert E